# Model Development Notebook

Bu notebook, haber metinleri için **kategori sınıflandırma, bilgi çıkarımı ve özetleme** işlemlerini gerçekleştiren modelin geliştirilme sürecini içerir.

In [1]:
import sys
import os
import torch
import transformers
import datasets
import nltk
import spacy
import yaml
sys.path.append(os.path.abspath(".."))  # notebooks klasörünün bir üst dizinini ekler
from src.multi_task_pipeline import load_category_model, multi_task_pipeline
from src.utils.zero_shot_classification import load_zero_shot_model, classify_zero_shot
from src.utils.evaluation_metrics import evaluate_metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


/Users/bugrabilgecelik/PycharmProjects/YZMD_BugraBilgeCelik/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0


In [2]:
# Model Yükleme
category_model, tokenizer, labels = load_category_model()

# Veri Setini Yükleme (AG News Test Verisi)
from datasets import load_dataset

dataset = load_dataset("ag_news", split="test").shuffle(seed=42).select(range(10))  # İlk 10 haberle test


In [3]:
# ** Model Testi (Inference) ve Karşılaştırma**
y_true, y_pred = [], []
rouge_scores, bleu_scores = [], []

for i, example in enumerate(dataset):
    text, true_category = example["text"], labels[example["label"]]
    result = multi_task_pipeline(text, category_model, tokenizer, labels)

    print(f"\n### Haber {i+1} ###")
    print("Gerçek Kategori:", true_category)
    print("Model Kategori tahmini:", result["category"])
    print("Model Çıktısı:")
    print(result)

    # **Kategori karşılaştırma için listeleri güncelle**
    y_true.append(true_category)
    y_pred.append(result["category"])

    # **Özetleme Metrikleri**
    summary_metrics = evaluate_metrics(text, result["summary"])
    rouge_scores.append(summary_metrics["ROUGE-L"])
    bleu_scores.append(summary_metrics["BLEU"])


### Haber 1 ###
Gerçek Kategori: Sports
Model Kategori tahmini: Sports
Model Çıktısı:
{'news_id': 'fb7be88f-c523-4170-a4a2-4e25278a3507', 'category': 'Sports', 'keywords': ['Australia', 'Indian', 'Wednesday'], 'entities': {'locations': ['Australia', 'Indian'], 'dates': ['2025-02-26', '2039-02-25']}, 'summary': "The Indian cricket board said on Wednesday it was making arrangements to broadcast next month's test series against Australia .Australia is under threat because of a raging TV rights dispute", 'confidence_scores': {'category': 0.9957192540168762, 'extraction': 0.9728596270084381}}

### Haber 2 ###
Gerçek Kategori: Business
Model Kategori tahmini: Business
Model Çıktısı:
{'news_id': 'f699bf9b-7fd9-4063-98a4-e06f53e571aa', 'category': 'Business', 'keywords': ['Drop', 'Thursday', 'Higher'], 'entities': {'locations': [], 'dates': ['2025-02-27']}, 'summary': 'A sharp drop in initial unemployment claims and bullish forecasts from Nokia and Texas Instruments sent stocks slightly highe

In [5]:
# **Model Performans Değerlendirmesi**
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

# **Özetleme metriklerinin ortalaması**
avg_rouge = sum(rouge_scores) / len(rouge_scores)
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print("\n### Genel Model Performansı ###")
print(f"Kategori Sınıflandırma - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
print(f"Özetleme - ROUGE-L: {avg_rouge:.4f}, BLEU: {avg_bleu:.4f}")


### Genel Model Performansı ###
Kategori Sınıflandırma - Accuracy: 0.8000, Precision: 0.8333, Recall: 0.8000, F1-score: 0.7600
Özetleme - ROUGE-L: 0.6690, BLEU: 0.4061


In [6]:
# Zero-Shot Modeli Yükle
zero_shot_model, zero_shot_tokenizer = load_zero_shot_model()

# Test İçin Bir Haber Örneği
text = "NASA is planning a new mission to explore Mars and gather data on its atmosphere."

# Zero-Shot Kategori Etiketleri
custom_labels = ["Politics", "Space" ,"Sports", "Business"]

# Zero-Shot Sınıflandırmayı Çalıştır
result = classify_zero_shot(text, custom_labels, zero_shot_model, zero_shot_tokenizer)

# Çıktıyı Yazdır
print("\n### Zero-Shot Sınıflandırma Sonucu ###")
print(f"🔹 Metin: {text}")
print(f"🔹 Tahmin Edilen Kategori: {result['label']}")
print(f"🔹 Güven Skoru: {result['confidence']:.4f}")

Device set to use mps:0



### Zero-Shot Sınıflandırma Sonucu ###
🔹 Metin: NASA is planning a new mission to explore Mars and gather data on its atmosphere.
🔹 Tahmin Edilen Kategori: Space
🔹 Güven Skoru: 0.6621


In [6]:
# Almanca Haber Metni Değerlendirme
german_news = "Jannik Sinner ist ein italienischer Tennisspieler. Es ist die Nummer 1 der Welt. Am 24. Februar wurde ihm ein Dopingverbot erteilt.."
print("\n### Almanca Haber Testi ###")
print("Girdi:", german_news)

german_result = multi_task_pipeline(german_news, category_model, tokenizer, labels)
print("Model Çıktısı:")
print(german_result)


### Almanca Haber Testi ###
Girdi: Jannik Sinner ist ein italienischer Tennisspieler. Es ist die Nummer 1 der Welt. Am 24. Februar wurde ihm ein Dopingverbot erteilt..
Model Çıktısı:
{'news_id': '1564b77a-4137-4271-81f8-843ac23762ef', 'category': 'Sports', 'keywords': ['Tennisspieler', 'Sinner', 'Jannik'], 'entities': {'locations': [], 'dates': ['2025-02-24']}, 'summary': 'Jannik Sinner ist ein italienischer Tennisspieler.Es ist die Nummer 1 der Welt. Am 24. Februar ihm ein Dopingverbot', 'confidence_scores': {'category': 0.9991621971130371, 'extraction': 0.9745810985565185}}


In [1]:
# **6️⃣ Web Arayüzünü Başlat (Gradio)**
import subprocess
import os

# **6️⃣ Web Arayüzünü Başlat (Gradio)**
print("\n🚀 **Gradio Web Arayüzü Başlatılıyor...**")
script_path = os.path.abspath(os.path.join(os.getcwd(), "../src/multi_task_web_demo.py"))
subprocess.run(["python", script_path])

print("\n✅ Web Arayüzü Aç: http://127.0.0.1:7860")


🚀 **Gradio Web Arayüzü Başlatılıyor...**


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0
Your max_length is set to 49, but your input_length is only 41. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0fddb691abd76aea80.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0fddb691abd76aea80.gradio.live


KeyboardInterrupt: 

### Sonuçlar:
- Modelin kategori sınıflandırma doğruluğu 7600 resmi test verisi için %93.12 civarında.
- Zero-Shot sınıflandırma, belirli konular için mantıklı tahminler yapabiliyor.
- Özetleme performansı BLEU ve ROUGE-L skorlarına göre değerlendirildi.

### Gelecek Çalışmalar:
- Veri seti artırımı ve model ince ayarları yapılabilir.
- Daha gelişmiş Zero-Shot modelleri denenebilir.
- Özetleme için farklı modeller test edilebilir.
